In [35]:
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
import keras
from keras import regularizers, optimizers
from keras.layers import Conv2D,Input,Dense,MaxPooling2D,BatchNormalization,ZeroPadding2D,Flatten,Dropout
from keras.models import Model
import numpy as np
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

In [36]:
from keras.callbacks import ReduceLROnPlateau, CSVLogger,EarlyStopping,ModelCheckpoint
import matplotlib.pyplot as plt
from scipy.misc import imread, imresize
from numpy.random import permutation

In [41]:
def le_net():
    model = Sequential()
    # first set of CONV => RELU => POOL
    model.add(Convolution2D(20, 5, 5, border_mode="same",
    input_shape=(3, 60, 60)))
    model.add(Dropout(0.2))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # second set of CONV => RELU => POOL
    model.add(Convolution2D(50, 5, 5, border_mode="same"))
    model.add(Dropout(0.2))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Dropout(0.2))
    model.add(Activation("relu"))
 
    # softmax classifier
    model.add(Dense(10))
    model.add(Dropout(0.2))
    model.add(Activation("softmax"))
        
    return model        

In [42]:
model = le_net()
model.summary()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (5, 5), padding="same", input_shape=(3, 60, 60...)`
  """
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (5, 5), padding="same")`
  # This is added back by InteractiveShellApp.init_path()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 20, 60, 60)        1520      
_________________________________________________________________
dropout_23 (Dropout)         (None, 20, 60, 60)        0         
_________________________________________________________________
activation_27 (Activation)   (None, 20, 60, 60)        0         
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 20, 30, 30)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 50, 30, 30)        25050     
_________________________________________________________________
dropout_24 (Dropout)         (None, 50, 30, 30)        0         
_________________________________________________________________
activation_28 (Activation)   (None, 50, 30, 30)        0         
__________

In [48]:
train_images = np.load('train_images_lenet.npy')
train_labels = np.load('train_labels_lenet.npy')

In [23]:
lr_reducer = ReduceLROnPlateau(factor = np.sqrt(0.1), cooldown=0, patience=2, min_lr=0.5e-6)
csv_logger = CSVLogger('Lenet.csv')
early_stopper = EarlyStopping(min_delta=0.001,patience=30)
model_checkpoint = ModelCheckpoint('Lenet.hdf5',monitor = 'val_loss', verbose = 1,save_best_only=True)

In [49]:
# Test pretrained model
train_images = np.array(train_images)
train_labels = np.array(train_labels)
mean = np.mean(train_images,axis=(0,1,2,3))
std = np.std(train_images,axis=(0,1,2,3))
train_images = (train_images-mean)/(std+1e-7)
num_classes = 10
train_labels = np_utils.to_categorical(train_labels,num_classes)

In [25]:
perm = permutation(len(train_images))
train_images = train_images[perm]
train_labels = train_labels[perm]
val_images = train_images[1:4800]
val_labels = train_labels[1:4800]
new_train= train_images[4800:]
new_labels = train_labels[4800:]

In [43]:
model.compile(loss='categorical_crossentropy',
        optimizer="Adam",
        metrics=['accuracy'])

In [27]:
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
datagen.fit(new_train)
model.fit_generator(datagen.flow(new_train, new_labels, batch_size=12),
                        steps_per_epoch=new_train.shape[0] // 12,
                        epochs=30,verbose=1,validation_data=(val_images,val_labels))

Epoch 1/30
1430/1430 [==============================] - 108s - loss: 1.1804 - acc: 0.6059 - val_loss: 0.6875 - val_acc: 0.7768
Epoch 2/30
1430/1430 [==============================] - 108s - loss: 0.7968 - acc: 0.7367 - val_loss: 0.4712 - val_acc: 0.8478
Epoch 3/30
1430/1430 [==============================] - 108s - loss: 0.6880 - acc: 0.7686 - val_loss: 0.4500 - val_acc: 0.8569
Epoch 4/30
1430/1430 [==============================] - 108s - loss: 0.6305 - acc: 0.7853 - val_loss: 0.3742 - val_acc: 0.8849
Epoch 5/30
1430/1430 [==============================] - 108s - loss: 0.6051 - acc: 0.7971 - val_loss: 0.2664 - val_acc: 0.9119
Epoch 6/30
1430/1430 [==============================] - 108s - loss: 0.5753 - acc: 0.8058 - val_loss: 0.2791 - val_acc: 0.9109
Epoch 7/30
1430/1430 [==============================] - 109s - loss: 0.5636 - acc: 0.8140 - val_loss: 0.2686 - val_acc: 0.9089
Epoch 8/30
1430/1430 [==============================] - 110s - loss: 0.5320 - acc: 0.8206 - val_loss: 0.3162 - 

In [29]:
#Training the model 
model.save_weights('lenet.h5')

In [44]:
#Loading pre-trained weights
model.load_weights('lenet.h5')

In [61]:
class_names = {0:'curl_virus',1:'late_blight',2:'early_blight',3:'spider_mites',4:'healthy',5:'septorial_leaf_spot',6:'target_spot',7:'mosaic_virus',8:'bacterial_spot',9:'leaf_mold'}

In [62]:
img = imresize(imread("Tomato/Tomato___Early_blight/1bb0101c-d0a4-41a5-85b9-6e8634d01a36___RS_Erly.B 9585.JPG", mode='RGB'),(60,60)).astype(np.float32)
img = (img-mean)/(std+1e-7)
img = np.expand_dims(img, axis=0)
out = model.predict(img) 
print out
print (np.argmax(out))
print(class_names[np.argmax(out)])


[[  3.40223694e-09   1.98008383e-05   9.99696732e-01   5.74970974e-11
    7.72399657e-16   2.80315354e-10   7.98787009e-16   2.09884723e-30
    2.83511530e-04   7.80457032e-09]]
2
early_blight


In [63]:
img = imresize(imread("Tomato/Tomato___healthy/0a9986e6-b629-4ff5-8aab-7488ea9b935b___RS_HL 9704.JPG", mode='RGB'),(60,60)).astype(np.float32)
img = (img-mean)/(std+1e-7)
img = np.expand_dims(img, axis=0)
out = model.predict(img) 
print out
print (np.argmax(out))
print(class_names[np.argmax(out)])



[[  8.73418612e-12   9.46001637e-06   4.22133356e-10   5.11822805e-07
    9.99881029e-01   6.79011100e-06   8.46701078e-05   1.30555435e-07
    9.63732358e-11   1.75554796e-05]]
4
healthy


In [46]:
labels = np.load('train_labels_lenet.npy')
labels_list = list(labels) 
print(type(labels_list))
print(len(labels_list))

<type 'list'>
18160


In [53]:
for i in range(10):
    print(str(i)+" "+str(labels_list.count(i)))

0 5357
1 1909
2 1000
3 1676
4 1591
5 1771
6 1404
7 373
8 2127
9 952
